In [7]:
#imports
from statsbombpy import sb
import mplsoccer

import pandas as pd
import numpy as np
import math

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from copy import deepcopy
from ast import literal_eval

In [8]:
#df settings
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 50)

In [9]:
data = pd.read_csv('cleaned_merged_seasons.csv',low_memory=False)

In [10]:
data

,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15T19:00:00Z,0,4,Chelsea,0,0,0,0,1,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,3,0,0,0.9,8.2,2016-08-13T14:00:00Z,15,17,Spurs,0,0,0,0,1,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,8,3,0,3.0,2.2,2016-08-14T15:00:00Z,60,9,Liverpool,0,0,0,0,1,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13T14:00:00Z,0,13,Southampton,0,0,0,0,1,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13T14:00:00Z,69,14,Stoke,0,0,0,0,1,0,2723,1.0,1.0,0.0,1,0,0,0,45,True,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98397,98397,2021-22,Wilfred Ndidi,MID,Leicester,0,0,0,0,0.0,216,377,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,16,Southampton,0,0,0,0,38,0,70366,1.0,4.0,0.0,0,-202,22,224,48,True,0,38
98398,98398,2021-22,Matt Ritchie,DEF,Newcastle,0,0,3,0,0.0,292,374,0,0,0.0,0.0,2022-05-22T15:00:00Z,1,5,Burnley,0,0,0,0,38,0,15999,2.0,1.0,0.0,1,143,396,253,49,False,0,38
98399,98399,2021-22,Nathan Redmond,MID,Southampton,0,0,5,0,0.0,336,377,3,0,0.0,0.0,2022-05-22T15:00:00Z,82,9,Leicester,0,0,0,0,38,0,13856,1.0,4.0,0.0,2,455,683,228,59,False,0,38
98400,98400,2021-22,Mathew Ryan,GK,Brighton,0,0,0,0,0.0,65,373,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,19,West Ham,0,0,0,0,38,0,587,1.0,3.0,0.0,0,-2,0,2,45,True,0,38


In [11]:
removing = data.groupby(by='name')['minutes'].agg(['sum'])
#removing players who have played 0 minutes throughout the data
removing = removing[removing['sum']==0]
removing.reset_index(inplace=True)

In [13]:
data['opp_team_name'].value_counts()

Man City          4956
Arsenal           4923
West Ham          4919
Spurs             4917
Chelsea           4917
Man Utd           4907
Southampton       4903
Liverpool         4897
Burnley           4890
Everton           4885
Crystal Palace    4875
Leicester         4860
Brighton          4514
Newcastle         4492
Wolves            3972
Watford           3701
Aston Villa       3278
Bournemouth       2545
Leeds             2445
West Brom         2172
Norwich           2104
Sheffield Utd     2094
Fulham            1872
Huddersfield      1258
Brentford         1229
Stoke              971
Swansea            952
Cardiff            700
Sunderland         422
Middlesbrough      421
Hull               411
Name: opp_team_name, dtype: int64

In [14]:
data.columns.values

array(['Unnamed: 0', 'season_x', 'name', 'position', 'team_x', 'assists',
       'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'fixture',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW'],
      dtype=object)

In [16]:
data.isna().sum()

Unnamed: 0               0
season_x                 0
name                     0
position                 0
team_x               49472
assists                  0
bonus                    0
bps                      0
clean_sheets             0
creativity               0
element                  0
fixture                  0
goals_conceded           0
goals_scored             0
ict_index                0
influence                0
kickoff_time             0
minutes                  0
opponent_team            0
opp_team_name            0
own_goals                0
penalties_missed         0
penalties_saved          0
red_cards                0
round                    0
saves                    0
selected                 0
team_a_score            49
team_h_score            49
threat                   0
total_points             0
transfers_balance        0
transfers_in             0
transfers_out            0
value                    0
was_home                 0
yellow_cards             0
G

In [17]:
#creating a dictinary for players names with their teams

main_dict={}
for index,row in data.iterrows():
    
    main_dict[row['name']] = row['team_x']
    
        
        
        

In [18]:
main_dict

{'Aaron Cresswell': 'West Ham',
 'Aaron Lennon': 'Burnley',
 'Aaron Ramsey': 'Aston Villa',
 'Abdoulaye Doucouré': 'Everton',
 'Adam Forshaw': 'Leeds',
 'Adam Lallana': 'Brighton',
 'Adrián San Miguel del Castillo': 'Liverpool',
 'Alex Iwobi': 'Everton',
 'Alex McCarthy': 'Southampton',
 'Alex Oxlade-Chamberlain': 'Liverpool',
 'Andreas Pereira': 'Man Utd',
 'Andrew Robertson': 'Liverpool',
 'Andre Gray': 'Watford',
 'Andros Townsend': 'Everton',
 'Andy Carroll': 'Newcastle',
 'Angelo Ogbonna': 'West Ham',
 'Anthony Martial': 'Man Utd',
 'Arthur Masuaku': 'West Ham',
 'Ashley Barnes': 'Burnley',
 'Ashley Young': 'Aston Villa',
 'Bamidele Alli': 'Everton',
 'Benjamin Chilwell': 'Chelsea',
 'Ben Davies': 'Spurs',
 'Ben Foster': 'Watford',
 'Ben Gibson': 'Norwich',
 'Ben Mee': 'Burnley',
 'Branislav Ivanovic': 'West Brom',
 'Callum Wilson': 'Newcastle',
 'Calum Chambers': 'Aston Villa',
 'Cameron Carter-Vickers': 'Spurs',
 'Charlie Austin': 'West Brom',
 'Cheikhou Kouyaté': 'Crystal Palac

In [19]:
#filling teams for players where team name was missing
for index,row in data.iterrows():
    if main_dict.get(row['name'])!=None:
        data.loc[index,'team_x']=main_dict[row['name']]
        
    if row['name']=='Aaron Ramsey':
        data.loc[index,'team_x']='Arsenal'

In [21]:
data.to_csv('only_played.csv')

In [22]:
data.shape

(98402, 38)

In [23]:
data.isna().sum()

Unnamed: 0            0
season_x              0
name                  0
position              0
team_x                0
assists               0
bonus                 0
bps                   0
clean_sheets          0
creativity            0
element               0
fixture               0
goals_conceded        0
goals_scored          0
ict_index             0
influence             0
kickoff_time          0
minutes               0
opponent_team         0
opp_team_name         0
own_goals             0
penalties_missed      0
penalties_saved       0
red_cards             0
round                 0
saves                 0
selected              0
team_a_score         49
team_h_score         49
threat                0
total_points          0
transfers_balance     0
transfers_in          0
transfers_out         0
value                 0
was_home              0
yellow_cards          0
GW                    0
dtype: int64